In [ ]:
!pip install ultralytics --quiet

from ultralytics import YOLO
import os
import glob
import zipfile
from IPython.display import display, Image

In [ ]:
dataset_path = "/kaggle/input/trash-filter-yolov8/DATASET"

train_images = os.path.join(dataset_path, "train/images")
train_labels = os.path.join(dataset_path, "train/labels")
val_images   = os.path.join(dataset_path, "valid/images")
val_labels   = os.path.join(dataset_path, "valid/labels")
test_images  = os.path.join(dataset_path, "test/images")

In [ ]:
yaml_file = "/kaggle/working/data.yaml"

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
results = model.train(
    data=yaml_file,
    epochs=50,
    imgsz=416,
    batch=16,
    save=True,   
    project="/kaggle/working/runs/train",
    name="trash_monitor",
)

In [ ]:
best_weights = "/kaggle/working/runs/train/trash_monitor/weights/best.pt"
print("Best weights saved at:", best_weights)

In [ ]:
predict_folder = "/kaggle/working/runs/detect"
os.makedirs(predict_folder, exist_ok=True)

In [ ]:
pred_model = YOLO(best_weights)

In [ ]:
pred_model.predict(
    source=test_images,
    conf=0.4,
    save=True,       
    save_txt=True,   
    project=predict_folder,
    name="predictions",
)

In [ ]:
zip_path = "/kaggle/working/predicted_results.zip"
folder_to_zip = os.path.join(predict_folder, "predictions")

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            zipf.write(os.path.join(root, file),
                       arcname=os.path.join(os.path.relpath(root, folder_to_zip), file))

print("Predicted images zipped at:", zip_path)

In [ ]:
sample_images = glob.glob(folder_to_zip + "/*.jpg")[:5]  # first 5 predicted images
for img_path in sample_images:
    display(Image(filename=img_path))